# Анализ чеков по штрихкодам


In [1]:
from sqlplusapi import SqlPlus
import pandas as pd
from os import path,listdir,environ
from functools import partial
import numpy as np
from subprocess import call

0

In [2]:
environ['NLS_LANG']='RUSSIAN_CIS.CL8MSWIN1251'
project_path=r'C:\Reports\barcodediscount\extra'
project=SqlPlus(project_path,printlog=True) #projectdirectory,delimiter='|',sql_plus_options_new=None,printlog=False

INFO:root: Project initialisation (2016-12-15 09:54:49; sqlplusapi.py:37)


In [3]:
sqlpath_1=r'C:\Scripts\SQL\barcode.sql'
sqlpath_2=r'C:\Scripts\SQL\barcodewithclassifier.sql'

query_params_1={'BegPeriod':"'01.12.2015'",
              'EndPeriod':"'01.12.2016'",
               'barfilter':"'=66'",
                'paytype':'23',
                'ending':'""'}
query_params_2={'BegPeriod':"'01.12.2015'",
                 'EndPeriod':"'01.12.2016'"
                 }
query_1=project.set_sql_query(query_params_1,sqlpath_1)
query_2=project.set_sql_query(query_params_2,sqlpath_2)
#project.printlines(query_2)#project.write_query(query=query)

Получаем Oracle id магазинов (из excel файла выбираем первый столбец и преобразуем в строку)

In [4]:
depart_codes_path='C:\Scripts\Dictionaries\departmentcodes.xlsx'
department_codes=list((read_excel(pd,depart_codes_path).ix[:,0]).apply(str))

conn_string=lambda department_name: "%s/%s@%s"% ("cash","suffolk","ORA%s.NTDOMAIN.PRK"%department_name)

## Запускаем асинхронные потоки для выгрузки данных

In [5]:
load_path_1=path.join(project.default_loaddirectory,'66')
load_path_2=path.join(project.default_loaddirectory,'other')

In [ ]:
%%time
load_path_1=path.join(project.default_loaddirectory,'66')
load_path_2=path.join(project.default_loaddirectory,'other')
project.prepareload(load_path_1)
project.prepareload(load_path_2)
#оборачиваем функцию выгрузки данных для распараллеливания
def worker(load_dir,sql_q,department_oraname):
    savepath_w=path.join(load_dir,'%s.csv'%department_oraname)
    project.FetchData(conn_string(department_oraname),savepath_w,sql_q)
    

worker_1=partial(worker,load_path_1,query_1)
worker_2=partial(worker,load_path_2,query_2)

project.ThreadPool(worker_1,department_codes)
project.ThreadPool(worker_2,department_codes)

### Склеиваем файлы
#### Note! в эксель выгонять данные нельзя(  из за лимита на число ячеек)

In [6]:
%%time
fetched_data_list=lambda savepath:list(map(lambda x:[path.splitext(x)[0],path.join(savepath,x)],
                           [f for f in listdir(savepath) if 'error_' not in f]))

df1=pd.concat(list(map(project.read_loaded_csv,fetched_data_list(load_path_1))))
df2=pd.concat(list(map(project.read_loaded_csv,fetched_data_list(load_path_2))))


Wall time: 6.43 s


In [7]:
df1.head()

,BARCODE,TOTAL_SUM,CORR_SUM,ID_DEPARTMENT,ID_PAYDESK,ID_HEADER,COUNT_UNIQ,filename
0,66,162,"-3,89",1697,25,516764,2,2X
1,77,237,"-12,95",1697,10,456487,1,2X
2,77,13930,"-41,14",1697,10,456632,14,2X
3,77,411,"-2,68",1697,18,183562,3,2X
4,77,2975,"-4,14",1697,18,183585,5,2X
